<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-27 13:08:28
-------------------
qualified stocks: 93
with latest results: 30
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.34 C
-------------------
Today PnL: 57.42 K (0.43%)
Current PnL: -14.86 L (-10.35%)
CY Booked + Current PnL: -8.37 L (-5.83%)
-------------------
Total profit:  3.83 L
Total loss:  -18.69 L
-------------------
Total Booked + Current PnL: 18.61 L (15.97%)
Total Booked PnL: 33.47 L (28.72%)
Curr Year Booked PnL: 6.49 L (4.83%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 71.74 L (53.36%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.8%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1291.83,1526.00,9.89,X-LC,76.34,162988.0,22179.0,3341.0,1.91,15.75,2.05,18.13,18.0,6.64,1.21,28.69,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.00,2.14,M-LC,54.84,102503.0,24141.0,9892.0,1.01,30.81,9.65,43.43,79.0,2.44,0.76,37.76,XY24,NTT,MISC
35,HINDALCO,651.95,761.55,-6.77,H-LC,70.97,111680.0,7368.0,10163.0,1.07,7.06,9.10,16.81,27.0,0.72,0.83,24.24,X5K,ATH,METALS
84,UJJIVANSFB,52.77,53.00,52.86,M-SC,84.95,132570.0,-9909.0,10526.0,0.00,-6.95,7.94,0.44,242.0,-0.94,0.99,56.47,OX40N,NTT,BANKS
85,UNIONBANK,123.87,163.00,-1.84,M-LC,78.49,172005.0,31165.0,13330.0,2.13,22.13,7.75,31.59,90.0,2.34,1.28,49.41,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.87,M-MC,60.22,181192.0,1384.0,111034.0,0.74,0.77,61.28,62.52,170.0,0.01,1.35,35.12,XR,NTT,BANKS
50,KPIGREEN,497.21,732.16,NaN,NaN,66.67,124576.0,-721.0,59934.0,-0.56,-0.58,48.11,47.25,137.0,-0.01,0.93,55.85,MH,ATH,POWER
34,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,34.41,151568.0,655.0,59794.0,1.23,0.43,39.45,40.06,102.0,0.01,1.13,23.97,AR,ATH,AUTO
47,JIOFIN,310.58,387.00,-0.96,M-LC,89.25,218750.0,1344.0,52150.0,3.03,0.62,23.84,24.61,94.0,0.03,1.63,55.53,XY24,BTT,FINANCE


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CERA,8421.60,8422.00,-14.08,X-SC,68.82,88751.0,-20730.0,20732.0,3.49,-18.93,23.36,0.00,68.0,-1.00,0.66,34.33,OX40N,NTT,CERAMICS
47,JIOFIN,310.58,387.00,-0.96,M-LC,89.25,218750.0,1344.0,52150.0,3.03,0.62,23.84,24.61,94.0,0.03,1.63,55.53,XY24,BTT,FINANCE
26,DIXON,14367.00,18940.15,-10.88,H-LC,23.66,117768.0,2832.0,33752.0,2.84,2.46,28.66,31.83,99.0,0.08,0.88,34.98,X40N,ATH,IT
43,INDUSINDBK,1354.50,1800.00,-33.43,L-MC,61.29,48969.0,-28238.0,53631.0,2.75,-36.57,109.52,32.89,252.0,-0.53,0.36,34.86,XR,NTT,BANKS
75,SURYODAY,189.78,240.00,55.70,H-SC,72.04,111003.0,-38923.0,78601.0,2.35,-25.96,70.81,26.46,143.0,-0.50,0.83,42.20,XR,NTT,BANKS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SAMMAANCAP,170.35,326.00,-169.79,M-SC,95.70,84312.0,-17898.0,111292.0,-1.99,-17.51,132.00,91.37,203.0,-0.16,0.63,36.87,XY25,NTT,FINANCE
60,RAJESHEXPO,517.65,518.00,1840.66,M-SC,38.71,52939.0,-84238.0,84332.0,-1.74,-61.41,159.30,0.07,232.0,-1.00,0.39,31.66,OX40N,NTT,JEWELLERY
41,INDIAMART,2327.09,4850.92,-47.17,H-SC,92.47,137562.0,14226.0,119541.0,-1.58,11.53,86.90,108.45,121.0,0.12,1.02,36.47,AR,ATH,MISC
16,BSOFT,628.85,844.81,-18.67,M-SC,64.52,76335.0,-33714.0,71503.0,-1.36,-30.64,93.67,34.34,200.0,-0.47,0.57,25.67,XR,ATH,IT
28,EASEMYTRIP,18.12,26.40,5.28,M-SC,6.45,101483.0,-75332.0,156132.0,-1.33,-42.60,153.85,45.70,189.0,-0.48,0.76,0.19,XY24,NTT,TRAVEL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,52.86,M-SC,84.95,132570.0,-9909.0,10526.0,0.00,-6.95,7.94,0.44,242.0,-0.94,0.99,56.47,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.0,-15.64,X-MC,36.56,88662.0,-13990.0,13982.0,0.93,-13.63,15.77,-0.00,58.0,-1.00,0.66,34.78,OX40N,NTT,IT
89,VIPIND,488.80,489.0,-954.05,H-SC,96.77,81315.0,-14001.0,14043.0,0.07,-14.69,17.27,0.04,153.0,-1.00,0.61,60.38,OX40N,NTT,MISC
76,SYMPHONY,1306.42,1306.0,-33.55,M-SC,1.08,141690.0,-29451.0,29401.0,0.11,-17.21,20.75,-0.03,188.0,-1.00,1.05,1.84,OX40N,NTT,DURABLES
42,INDIGOPNTS,1407.73,1408.0,119.87,H-SC,86.02,144212.0,-30347.0,30385.0,-0.25,-17.38,21.07,0.02,128.0,-1.00,1.07,25.05,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-17.44,X-LC,30.11,268999.0,-26935.0,120834.0,0.20,-9.10,44.92,31.73,1.0,-0.22,2.00,7.67,X40,BTT,IT
44,INFY,1461.46,2275.00,-9.91,X-LC,56.99,288709.0,27108.0,118515.0,-0.19,10.36,41.05,55.67,2.0,0.23,2.15,17.10,X40,BTT,IT
36,HINDUNILVR,2413.81,2723.00,-18.24,X-LC,22.58,256984.0,-13363.0,48005.0,0.60,-4.94,18.68,12.81,6.0,-0.28,1.91,7.48,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,16.68,X-LC,69.89,267513.0,38280.0,28998.0,-0.21,16.70,10.84,29.35,7.0,1.32,1.99,27.11,XY25,ATH,FMCG
57,NESTLEIND,2268.60,2755.00,-9.68,X-LC,31.18,286650.0,21224.0,35688.0,0.76,8.00,12.45,21.44,9.0,0.59,2.13,14.69,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.0,-28.73,M-SC,90.32,207946.0,36325.0,125745.0,0.50,21.17,60.47,94.43,196.0,0.29,1.55,53.48,XR,NTT,DURABLES
85,UNIONBANK,123.87,163.0,-1.84,M-LC,78.49,172005.0,31165.0,13330.0,2.13,22.13,7.75,31.59,90.0,2.34,1.28,49.41,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.14,M-LC,54.84,102503.0,24141.0,9892.0,1.01,30.81,9.65,43.43,79.0,2.44,0.76,37.76,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,100.99,M-MC,53.76,228613.0,3651.0,164007.0,-0.64,1.62,71.74,74.53,186.0,0.02,1.70,32.73,XY24,BTT,STEEL
18,CAMS,3643.00,5250.99,6.44,H-SC,79.57,119616.0,17612.0,27416.0,1.41,17.27,22.92,44.14,124.0,0.64,0.89,36.31,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.14,M-LC,54.84,102503.0,24141.0,9892.0,1.01,30.81,9.65,43.43,79.0,2.44,0.76,37.76,XY24,NTT,MISC
34,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,34.41,151568.0,655.0,59794.0,1.23,0.43,39.45,40.06,102.0,0.01,1.13,23.97,AR,ATH,AUTO
47,JIOFIN,310.58,387.00,-0.96,M-LC,89.25,218750.0,1344.0,52150.0,3.03,0.62,23.84,24.61,94.0,0.03,1.63,55.53,XY24,BTT,FINANCE


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-28.73,M-SC,90.32,207946.0,36325.0,125745.0,0.50,21.17,60.47,94.43,196.0,0.29,1.55,53.48,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-3.81,M-LC,77.42,164753.0,13808.0,95656.0,-1.04,9.15,58.06,72.51,69.0,0.14,1.23,13.49,XR,NTT,IT
41,INDIAMART,2327.09,4850.92,-47.17,H-SC,92.47,137562.0,14226.0,119541.0,-1.58,11.53,86.90,108.45,121.0,0.12,1.02,36.47,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-35.66,H-SC,74.19,102327.0,1823.0,28815.0,-0.27,1.81,28.16,30.49,97.0,0.06,0.76,34.81,MH,ATH,METALS
34,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,34.41,151568.0,655.0,59794.0,1.23,0.43,39.45,40.06,102.0,0.01,1.13,23.97,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-51.71,L-SC,93.55,71654.0,-41895.0,81944.0,-0.93,-36.90,114.36,35.27,261.0,-0.51,0.53,75.38,XR,NTT,HOTELS
54,MASFIN,326.60,402.39,-17.80,H-SC,75.27,93390.0,-4590.0,27326.0,0.76,-4.68,29.26,23.21,141.0,-0.17,0.69,35.45,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,3.55,X-SC,98.92,96466.0,-4497.0,99032.0,-1.19,-4.45,102.66,93.63,65.0,-0.05,0.72,43.68,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.14,M-LC,54.84,102503.0,24141.0,9892.0,1.01,30.81,9.65,43.43,79.0,2.44,0.76,37.76,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-12.13,H-LC,50.54,108525.0,4959.0,15747.0,1.88,4.79,14.51,20.00,31.0,0.31,0.81,19.54,X40N,BTT,HEALTHCARE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7233.33,M-SC,100.00,80487.0,-13323.0,89944.0,-0.17,-14.20,111.75,81.67,240.0,-0.15,0.60,55.87,XR,NTT,CERAMICS
4,ALKYLAMINE,2347.98,4546.37,3.55,X-SC,98.92,96466.0,-4497.0,99032.0,-1.19,-4.45,102.66,93.63,65.0,-0.05,0.72,43.68,SR,ATH,CHEMICALS
48,JPPOWER,18.73,26.20,-32.62,L-SC,97.85,136541.0,-5301.0,61867.0,0.33,-3.74,45.31,39.88,259.0,-0.09,1.02,38.37,XY24,NTT,POWER
89,VIPIND,488.80,489.00,-954.05,H-SC,96.77,81315.0,-14001.0,14043.0,0.07,-14.69,17.27,0.04,153.0,-1.00,0.61,60.38,OX40N,NTT,MISC
66,SAMMAANCAP,170.35,326.00,-169.79,M-SC,95.70,84312.0,-17898.0,111292.0,-1.99,-17.51,132.00,91.37,203.0,-0.16,0.63,36.87,XY25,NTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.22
1,25,42.45
2,50,71.09


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.55
LC    37.26
MC    21.57
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.25
XY25     13.39
X40      12.38
X40N     10.64
XR       10.35
OX40N     8.25
AR        7.89
MH        1.69
X5K       1.50
X200      1.44
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.39
X-LC    15.38
M-SC    13.93
H-LC    12.45
M-LC     8.34
H-MC     8.12
X-MC     7.45
M-MC     5.64
X-SC     2.76
L-SC     2.47
L-LC     1.09
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,-1.52,35.49
IT,11.97,-10.49,65.33
BANKS,8.00,-6.12,49.43
FINANCE,7.91,-13.34,54.36
MISC,6.64,-21.00,68.20
PAINTS,5.65,-14.91,37.57
ELECTRICAL,5.06,-3.33,48.09
HEALTHCARE,4.51,-6.29,37.34
AUTO,4.42,-13.94,64.04


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2731600.0,25
XR,1068083.0,13
AR,1009426.0,9
X40,655001.0,9
X40N,501790.0,10
XY25,485461.0,9
OX40N,319841.0,11
SR,181395.0,2
MH,88749.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1889177.0,19
M-SC,1455572.0,18
X-LC,596217.0,9
M-MC,586917.0,5
H-LC,580678.0,11
H-MC,428512.0,6
X-MC,415546.0,6
M-LC,323695.0,6
L-SC,265089.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,865139.0,6
M-SC,XY24,602068.0,5
H-SC,AR,519495.0,3
M-MC,XY24,384088.0,3
M-SC,XR,328042.0,4
X-LC,X40,317307.0,3
H-LC,X40N,231534.0,5
H-MC,XY24,227535.0,2
H-SC,XR,213241.0,3


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
